In [3]:
# reading from s3
import boto3
import json
import pandas as pd

s3 = boto3.resource('s3')
bucket = s3.Bucket('iot-project-ylchua2')

# Iterates through all the objects, doing the pagination for you. Each obj
# is an ObjectSummary, so it doesn't contain the body. You'll need to call
# get to get the whole body.
for obj in bucket.objects.all():
    key = obj.key
    body = obj.get()['Body'].read()

    #df = pd.read_json(body.decode("utf-8"))
    df = pd.read_csv('scenario_1.csv')

In [4]:
df.head()

,Unnamed: 0,nearby,devices,datetime,location
0,0,1,iPhone 8 Plus,2022-05-11 09:58:19.762204,Beacon1
1,1,1,House Keys,2022-05-11 09:58:19.771136,Beacon1
2,2,1,Wallet,2022-05-11 09:58:19.776188,Beacon1
3,3,1,iPhone 8 Plus,2022-05-11 09:58:24.835336,Beacon1
4,4,1,House Keys,2022-05-11 09:58:24.841830,Beacon1


In [5]:
df.tail()

,Unnamed: 0,nearby,devices,datetime,location
295,295,0,House Keys,2022-05-11 10:08:58.121679,Beacon1
296,296,1,Wallet,2022-05-11 10:08:58.127359,Beacon1
297,297,0,iPhone 8 Plus,2022-05-11 10:09:03.195392,Beacon1
298,298,0,House Keys,2022-05-11 10:09:03.203322,Beacon1
299,299,1,Wallet,2022-05-11 10:09:03.208116,Beacon1


In [6]:
# aws sns
sns = boto3.resource("sns")

def publish_text_message(phone_number, message):
    """
    Publishes a text message directly to a phone number without need for a
    subscription.

    :param phone_number: The phone number that receives the message.
    :param message: The message to send.
    :return: The ID of the message.
    """
    response = sns.meta.client.publish(
        PhoneNumber=phone_number, Message=message)
    message_id = response['MessageId']
    return message_id

#publish_text_message("+65910491", "Testing sms 1234")

In [7]:
# analyze the data

# check for phone location
phone_present = df.loc[df['devices']=='iPhone 8 Plus'].iloc[-1].nearby
#phone_present 

# phone present
if phone_present == True:
    
    # check for items missing
    if (df.groupby(['devices']).last().nearby == False).any():
        #print(df.loc[df['nearby'] == 0].groupby(['devices']).last().index[0])
        #print(df.loc[df['nearby'] == 0].groupby(['devices']).last().location[0])
        msg = 'Item ' + df.loc[df['nearby'] == 0].groupby(['devices']).last().index[0] + \
                ' is missing from location ' + \
                df.loc[df['nearby'] == 0].groupby(['devices']).last().location[0] + '!!!'
        print(msg)
        #publish_text_message("+65910491", msg)
    else:
        device_names = [x+' are@' for x in df.groupby(['devices']).last().index]
        device_locs = [x+',' for x in df.groupby(['devices']).last().location]
        
        result = [None]*(len(device_names)+len(device_locs))
        result[::2] = device_names 
        result[1::2] = device_locs
        msg = 'Your tracked items are ' + ' '.join(result)
        print(msg)
        #publish_text_message("+65910491", msg)
# user left
else:
    # check for anything user didnt take
    #print(df.tail(30).loc[df['nearby'] == 1].groupby(['devices']).last().index[0])
    #print(df.tail(30).loc[df['nearby'] == 1].groupby(['devices']).last().location[0])
    msg = 'Did you forget to bring your ' + \
            df.tail(30).loc[df['nearby'] == 1].groupby(['devices']).last().index[0] + \
            '? It is still at ' + df.tail(30).loc[df['nearby'] == 1].groupby(['devices']).last().location[0]
    print(msg)
    #publish_text_message("+65910491", msg)

Did you forget to bring your Wallet? It is still at Beacon1
